# Introduction
In this workbook, we will build a neural network from scratch.  You will be introduced to PyTorch and the power of neural networks.  Lets get started!

Lets start by importing the software libraries we will need to build our neural network

In [1]:
# Import PyTorch libraries
import torch
import torchvision
from torch import nn 

### Verify Libraries are Loaded
Verify the libraries have been loaded

In [2]:
# Verify PyTorch libraries have been loaded
torch.__version__

'1.7.1'

In [3]:
import csv
lemonData=[]
with open('lemons.csv', newline='') as csvfile:
    datareader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in datareader:
        lemonData.append(row)

In [4]:
import pandas as pd
df = pd.read_csv('lemons.csv', nrows=1) # just get the columns
columns = df.columns.tolist() # get the columns
print (columns)
cols_to_use = columns[:len(columns)-1] # drop the last one
xDF = pd.read_csv('lemons.csv', usecols=cols_to_use)
print (xDF.head())
yDF = pd.read_csv('lemons.csv', usecols=['NumberLemonsSold'])
print (yDF.head())

['Weekend', 'Afternoon', 'Sunny', 'Warm', 'Sign', 'CuteDog', 'Ballons', 'Discount', 'FreshlyPicked', 'Ice', 'CleanHouse', 'DogWalked', 'NumberLemonsSold']
   Weekend  Afternoon  Sunny  Warm  Sign  CuteDog  Ballons  Discount  \
0        1          1      1     0     1        0        0         0   
1        1          0      1     0     1        0        0         0   
2        0          0      1     0     0        1        1         0   
3        0          0      1     1     0        1        0         0   
4        1          1      1     0     0        1        0         0   

   FreshlyPicked  Ice  CleanHouse  DogWalked  
0              0    1           0          1  
1              0    0           1          0  
2              0    0           0          0  
3              1    1           1          1  
4              1    1           0          0  
   NumberLemonsSold
0                10
1                 7
2                 0
3                 0
4                14


In [5]:
x = torch.tensor(xDF.values,dtype=torch.float)
y = torch.tensor(yDF.values,dtype=torch.float)
print (x)
print (y)

tensor([[1., 1., 1.,  ..., 1., 0., 1.],
        [1., 0., 1.,  ..., 0., 1., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 1., 1.],
        [1., 0., 0.,  ..., 0., 0., 1.],
        [0., 0., 0.,  ..., 0., 1., 1.]])
tensor([[10.],
        [ 7.],
        [ 0.],
        ...,
        [ 0.],
        [ 2.],
        [ 0.]])


In [6]:
class MyFirstNetwork(nn.Module):
    def __init__(self):
        super(MyFirstNetwork, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(12,30),
            nn.ReLU(),
            nn.Linear(30,1)
        )
    def forward(self, x):
        logits = self.network(x)
        return logits


In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('A {} device was found for processing'.format(device))

A cuda device was found for processing


In [8]:
model = MyFirstNetwork()

In [9]:
model.to(device)
x = x.to(device)
y = y.to(device)

In [10]:
x

tensor([[1., 1., 1.,  ..., 1., 0., 1.],
        [1., 0., 1.,  ..., 0., 1., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 1., 1.],
        [1., 0., 0.,  ..., 0., 0., 1.],
        [0., 0., 0.,  ..., 0., 1., 1.]], device='cuda:0')

In [11]:
import torch.optim as optim
criterion = torch.nn.MSELoss()  #reduction='sum' maybe?
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
for epoch in range(5):
    totalLoss = 0
    for i in range(len(x)):
        # Forward pass: Compute predicted y by passing x to the model
        
        optimizer.zero_grad()
        ypred = model(x[i])
        loss = criterion(ypred, y[i])
        totalLoss+=loss.item()
        loss.backward()
        optimizer.step()
    print (totalLoss)


9029.218535295988
58.32629636172527
26.8369252670728
15.03876121192231
9.289065502303796


In [12]:
# Data that affects the number of lemons sold in one day
weekend = 1
afternoon = 1
sunny = 1
warm = 0

sign = 1
cutedog = 0
ballons = 1
discount = 1
freshlypicked = 1
ice = 1

# Data that usually does not affect lemons sold
houseclean = 1
dogwentwalk = 1

numlemonssold = 0
if weekend:
    numlemonssold = afternoon + sunny*5 + warm*10 + sign*2 + cutedog + ballons+discount+freshlypicked*5 + ice*2

print ("Actual ", numlemonssold)


Actual  17


In [13]:
model.to('cpu')
x = torch.tensor([weekend, afternoon,sunny,warm, sign, cutedog, 
                 ballons, discount, freshlypicked, ice, houseclean, 
                 dogwentwalk],dtype=float)
y=model(x.float())
print("Model Predicted: ",y.item())

Model Predicted:  16.99831199645996
